# 03. SVM Hard Margin 완전 정복

## 목표
- 왜 margin을 1로 정규화하는지 기하학적으로 이해
- Lagrangian multiplier 방법 완전 마스터
- Primal problem에서 Dual problem 유도
- KKT 조건의 의미와 역할 이해

---

## 1. SVM의 직관

### 1.1 문제 설정

**Linear classification**: 데이터를 선형으로 분리하는 초평면(hyperplane)을 찾기

- 데이터: $(x_1, y_1), \ldots, (x_n, y_n)$ where $x_i \in \mathbb{R}^d$, $y_i \in \{-1, +1\}$
- 초평면: $w^T x + b = 0$
- 분류 규칙: $\text{sign}(w^T x + b)$

### 1.2 여러 개의 분류 초평면

선형 분리 가능한 데이터라면 **무수히 많은** 초평면이 존재합니다.

**질문**: 어떤 초평면이 가장 좋을까?

**SVM의 답**: **Margin이 최대인** 초평면!

### 1.3 Margin의 정의

**Geometric Margin**: 가장 가까운 데이터 포인트와 초평면 사이의 거리

점 $x_0$에서 초평면 $w^T x + b = 0$까지의 거리:

$$\text{distance} = \frac{|w^T x_0 + b|}{||w||}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import seaborn as sns

sns.set_style('whitegrid')
np.random.seed(42)

# 2D 데이터 생성
# Class +1
X_pos = np.random.randn(20, 2) + np.array([2, 2])
y_pos = np.ones(20)

# Class -1
X_neg = np.random.randn(20, 2) + np.array([-2, -2])
y_neg = -np.ones(20)

X = np.vstack([X_pos, X_neg])
y = np.hstack([y_pos, y_neg])

# 여러 개의 분리 초평면
hyperplanes = [
    {'w': np.array([1, 1]), 'b': 0},  # Good
    {'w': np.array([1, 0.5]), 'b': 0.5},  # Bad (too close to positive)
    {'w': np.array([0.5, 1]), 'b': -0.5},  # Bad (too close to negative)
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (ax, hp) in enumerate(zip(axes, hyperplanes)):
    # 데이터 플롯
    ax.scatter(X_pos[:, 0], X_pos[:, 1], c='red', marker='o', s=100, label='Class +1', edgecolors='k')
    ax.scatter(X_neg[:, 0], X_neg[:, 1], c='blue', marker='s', s=100, label='Class -1', edgecolors='k')
    
    # 초평면 그리기
    w = hp['w']
    b = hp['b']
    
    xx = np.linspace(-5, 5, 100)
    yy = -(w[0] * xx + b) / w[1]
    ax.plot(xx, yy, 'g-', linewidth=2, label='Decision boundary')
    
    # Margin 계산
    margins = []
    for xi, yi in zip(X, y):
        distance = np.abs(w @ xi + b) / np.linalg.norm(w)
        margins.append(distance)
    
    min_margin = np.min(margins)
    
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_title(f'Hyperplane {idx+1}\nMargin = {min_margin:.2f}')
    ax.legend()
    ax.grid(alpha=0.3)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("관찰: 첫 번째 초평면이 가장 큰 margin을 가집니다!")

## 2. 왜 Margin = 1로 정규화하는가?

### 2.1 스케일 불변성 (Scale Invariance)

초평면 $w^T x + b = 0$을 $c(w^T x + b) = 0$ (단, $c > 0$)으로 스케일해도 **같은 초평면**입니다!

- $(w, b)$와 $(cw, cb)$는 같은 초평면을 정의
- 따라서 $(w, b)$에 **자유도**가 있음

### 2.2 정규화 (Normalization)

이 자유도를 이용하여 **가장 가까운 점**에서:

$$|w^T x_{\text{closest}} + b| = 1$$

로 **정규화**할 수 있습니다.

### 2.3 결과: Canonical Form

정규화 후:
- Positive class: $w^T x_i + b \geq +1$ for $y_i = +1$
- Negative class: $w^T x_i + b \leq -1$ for $y_i = -1$

합쳐서:
$$y_i(w^T x_i + b) \geq 1 \quad \forall i$$

**Margin** (초평면에서 서포트 벡터까지 거리):
$$\gamma = \frac{1}{||w||}$$

### 2.4 왜 1인가?

**답**: 임의의 선택! 0.5나 2로 해도 되지만:
- 1이 가장 **간단**
- 수식이 **깔끔**해짐
- 전체 margin = $\frac{2}{||w||}$ (양쪽 합)

**핵심**: 스케일 불변성 덕분에 어떤 값으로 정규화해도 **본질적으로 같은 문제**입니다!

In [ ]:
# 스케일 불변성 시각화

# 원래 초평면
w = np.array([1, 1])
b = 0

# 스케일된 버전들
scales = [0.5, 1, 2, 3]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 데이터
X_pos_simple = np.array([[2, 1], [3, 2]])
X_neg_simple = np.array([[-2, -1], [-3, -2]])

# 같은 초평면
ax = axes[0]
ax.scatter(X_pos_simple[:, 0], X_pos_simple[:, 1], c='red', marker='o', s=150, edgecolors='k', label='Class +1')
ax.scatter(X_neg_simple[:, 0], X_neg_simple[:, 1], c='blue', marker='s', s=150, edgecolors='k', label='Class -1')

xx = np.linspace(-4, 4, 100)
colors = plt.cm.rainbow(np.linspace(0, 1, len(scales)))

for scale, color in zip(scales, colors):
    w_scaled = scale * w
    b_scaled = scale * b
    yy = -(w_scaled[0] * xx + b_scaled) / w_scaled[1]
    ax.plot(xx, yy, color=color, linewidth=2, label=f'c={scale}')

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_title('스케일이 달라도 같은 초평면!')
ax.legend()
ax.grid(alpha=0.3)
ax.set_aspect('equal')

# Margin 변화
ax = axes[1]
margins = [1 / (scale * np.linalg.norm(w)) for scale in scales]
ax.bar(range(len(scales)), margins, color=colors, edgecolor='black', linewidth=2)
ax.set_xlabel('스케일 c')
ax.set_ylabel('Margin (1/||cw||)')
ax.set_title('스케일에 따른 Margin 변화')
ax.set_xticks(range(len(scales)))
ax.set_xticklabels([f'c={s}' for s in scales])
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("=== 스케일 불변성 ===")
for scale in scales:
    w_scaled = scale * w
    b_scaled = scale * b
    margin = 1 / np.linalg.norm(w_scaled)
    print(f"c = {scale}: w = {w_scaled}, margin = {margin:.4f}")
print("\n→ 스케일이 커질수록 ||w||도 커져서 margin이 작아짐")
print("→ Canonical form (margin=1 at support vectors)으로 정규화하면 ||w||이 고정됨!")

## 3. Primal Problem

### 3.1 최적화 문제 정식화

**목표**: Margin $\gamma = \frac{1}{||w||}$을 최대화

이것은 $||w||$를 최소화하는 것과 같습니다.

편의상 $\frac{1}{2}||w||^2$을 최소화합니다 (미분이 깔끔):

$$\begin{align}
\min_{w, b} \quad & \frac{1}{2}||w||^2 \\
\text{subject to} \quad & y_i(w^T x_i + b) \geq 1, \quad i = 1, \ldots, n
\end{align}$$

이것이 **Primal Problem**입니다!

### 3.2 문제의 성질

- **Convex optimization**: 목적 함수가 convex, 제약 조건이 linear
- **Quadratic Programming (QP)**: 목적 함수가 이차식
- **해가 유일**: Strictly convex하므로 global minimum이 유일

## 4. Lagrangian과 KKT 조건

### 4.1 Lagrangian 함수

제약 조건을 Lagrange multiplier $\alpha_i \geq 0$로 통합:

$$\mathcal{L}(w, b, \alpha) = \frac{1}{2}||w||^2 - \sum_{i=1}^{n} \alpha_i [y_i(w^T x_i + b) - 1]$$

**왜 마이너스?** 
- 제약: $y_i(w^T x_i + b) - 1 \geq 0$
- 위반하면 penalty: $-\alpha_i \times (\text{negative value}) = \text{positive penalty}$

### 4.2 Primal Formulation

$$\min_{w, b} \max_{\alpha \geq 0} \mathcal{L}(w, b, \alpha)$$

**해석**:
- 제약을 만족하면: $\max_{\alpha} \mathcal{L} = \frac{1}{2}||w||^2$ ($\alpha_i = 0$에서 최대)
- 제약을 위반하면: $\max_{\alpha} \mathcal{L} = +\infty$ ($\alpha_i \to \infty$)

### 4.3 Dual Formulation

Convex optimization에서 **Dual problem**:

$$\max_{\alpha \geq 0} \min_{w, b} \mathcal{L}(w, b, \alpha)$$

**Strong duality** (Slater's condition 만족):
$$\min_{w, b} \max_{\alpha} \mathcal{L} = \max_{\alpha} \min_{w, b} \mathcal{L}$$

따라서 두 문제의 **최적값이 같습니다**!

## 5. Dual Problem 유도

### 5.1 Step 1: $\mathcal{L}$을 $w, b$에 대해 최소화

$$\frac{\partial \mathcal{L}}{\partial w} = w - \sum_{i=1}^{n} \alpha_i y_i x_i = 0$$

$$\Rightarrow w = \sum_{i=1}^{n} \alpha_i y_i x_i$$

$$\frac{\partial \mathcal{L}}{\partial b} = -\sum_{i=1}^{n} \alpha_i y_i = 0$$

$$\Rightarrow \sum_{i=1}^{n} \alpha_i y_i = 0$$

### 5.2 Step 2: 대입해서 $\alpha$만의 함수로

$w = \sum_i \alpha_i y_i x_i$를 $\mathcal{L}$에 대입:

$$\begin{align}
\mathcal{L} &= \frac{1}{2}w^T w - \sum_i \alpha_i y_i w^T x_i - b\sum_i \alpha_i y_i + \sum_i \alpha_i \\
&= \frac{1}{2}w^T w - w^T w - 0 + \sum_i \alpha_i \quad (\text{since } \sum_i \alpha_i y_i = 0, w^T x_i = \sum_j \alpha_j y_j x_j^T x_i) \\
&= -\frac{1}{2}w^T w + \sum_i \alpha_i \\
&= -\frac{1}{2}\sum_{i,j} \alpha_i \alpha_j y_i y_j x_i^T x_j + \sum_i \alpha_i
\end{align}$$

### 5.3 Dual Problem

$$\begin{align}
\max_{\alpha} \quad & \sum_{i=1}^{n} \alpha_i - \frac{1}{2}\sum_{i,j=1}^{n} \alpha_i \alpha_j y_i y_j x_i^T x_j \\
\text{subject to} \quad & \alpha_i \geq 0, \quad i = 1, \ldots, n \\
& \sum_{i=1}^{n} \alpha_i y_i = 0
\end{align}$$

또는 최소화 형태로:

$$\begin{align}
\min_{\alpha} \quad & \frac{1}{2}\sum_{i,j=1}^{n} \alpha_i \alpha_j y_i y_j x_i^T x_j - \sum_{i=1}^{n} \alpha_i \\
\text{subject to} \quad & \alpha_i \geq 0, \quad i = 1, \ldots, n \\
& \sum_{i=1}^{n} \alpha_i y_i = 0
\end{align}$$

In [ ]:
# Dual problem을 직접 풀어보기 (2D 예제)
from scipy.optimize import minimize

# 간단한 데이터
X_train = np.array([
    [1, 2],
    [2, 3],
    [-1, -2],
    [-2, -3]
])
y_train = np.array([1, 1, -1, -1])

n_samples = len(y_train)

# Gram matrix: K[i,j] = x_i^T x_j
K = X_train @ X_train.T

# Dual objective: 1/2 * α^T Q α - 1^T α
# where Q[i,j] = y_i * y_j * K[i,j]
Q = np.outer(y_train, y_train) * K

def dual_objective(alpha):
    """Dual objective (to minimize)"""
    return 0.5 * alpha @ Q @ alpha - np.sum(alpha)

def dual_gradient(alpha):
    """Gradient of dual objective"""
    return Q @ alpha - np.ones(n_samples)

# 제약 조건: sum(α_i * y_i) = 0
constraints = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y_train)}

# 경계: α_i >= 0
bounds = [(0, None) for _ in range(n_samples)]

# 초기값
alpha_init = np.ones(n_samples)

# 최적화
result = minimize(
    dual_objective,
    alpha_init,
    method='SLSQP',
    jac=dual_gradient,
    bounds=bounds,
    constraints=constraints
)

alpha_opt = result.x

print("=== Dual Problem 풀이 ===")
print(f"최적 α: {alpha_opt}")
print(f"Objective value: {result.fun:.4f}")

# w 복원: w = Σ α_i y_i x_i
w_opt = np.sum(alpha_opt[:, np.newaxis] * y_train[:, np.newaxis] * X_train, axis=0)
print(f"\n복원된 w: {w_opt}")

# Support vectors (α > 0인 점들)
sv_indices = np.where(alpha_opt > 1e-5)[0]
print(f"\nSupport vector indices: {sv_indices}")
print(f"Support vectors:")
for idx in sv_indices:
    print(f"  x_{idx} = {X_train[idx]}, y_{idx} = {y_train[idx]}, α_{idx} = {alpha_opt[idx]:.4f}")

# b 계산: y_i(w^T x_i + b) = 1 for support vectors
b_values = []
for idx in sv_indices:
    b_val = y_train[idx] - w_opt @ X_train[idx]
    b_values.append(b_val)
b_opt = np.mean(b_values)
print(f"\n복원된 b: {b_opt:.4f}")

# 검증
print("\n=== 검증 ===")
for i in range(n_samples):
    decision = w_opt @ X_train[i] + b_opt
    margin = y_train[i] * decision
    print(f"x_{i}: y * (w^T x + b) = {margin:.4f} (should be >= 1)")

## 6. KKT 조건 (Karush-Kuhn-Tucker Conditions)

### 6.1 KKT 조건이란?

제약이 있는 최적화 문제의 **필요충분조건**:

1. **Stationarity** (정류조건):
   $$\nabla_w \mathcal{L} = 0, \quad \nabla_b \mathcal{L} = 0$$

2. **Primal feasibility** (주문제 실행가능성):
   $$y_i(w^T x_i + b) \geq 1, \quad \forall i$$

3. **Dual feasibility** (쌍대 실행가능성):
   $$\alpha_i \geq 0, \quad \forall i$$

4. **Complementary slackness** (상보성):
   $$\alpha_i [y_i(w^T x_i + b) - 1] = 0, \quad \forall i$$

### 6.2 Complementary Slackness의 의미

각 데이터 포인트 $i$에 대해 **둘 중 하나**가 성립:

1. $\alpha_i = 0$: 제약이 inactive (margin 밖, 분류에 영향 없음)
2. $y_i(w^T x_i + b) = 1$: 제약이 active (margin 위, **support vector**)

**핵심**:
- $\alpha_i > 0$ ⇔ $x_i$는 support vector
- 대부분의 $\alpha_i = 0$ ⇒ **Sparse solution**!

### 6.3 Geometric Interpretation

- **Support vectors**: Margin 경계에 정확히 위치한 점들
- **Non-support vectors**: Margin 밖에 있는 점들 ($\alpha_i = 0$)
- 초평면은 **오직 support vectors만**으로 결정됩니다!

In [ ]:
# KKT 조건 시각화

# Support vectors 하이라이트
plt.figure(figsize=(10, 8))

# 모든 데이터
for i in range(n_samples):
    if i in sv_indices:
        # Support vector
        if y_train[i] == 1:
            plt.scatter(X_train[i, 0], X_train[i, 1], c='red', marker='o', s=400, 
                       edgecolors='black', linewidths=3, label='SV (+1)' if i == sv_indices[0] else '')
        else:
            plt.scatter(X_train[i, 0], X_train[i, 1], c='blue', marker='s', s=400,
                       edgecolors='black', linewidths=3, label='SV (-1)' if y_train[i] == -1 and i == sv_indices[1] else '')
    else:
        # Non-support vector
        if y_train[i] == 1:
            plt.scatter(X_train[i, 0], X_train[i, 1], c='pink', marker='o', s=200,
                       edgecolors='gray', linewidths=2, alpha=0.5)
        else:
            plt.scatter(X_train[i, 0], X_train[i, 1], c='lightblue', marker='s', s=200,
                       edgecolors='gray', linewidths=2, alpha=0.5)

# Decision boundary
xx = np.linspace(-4, 4, 100)
yy = -(w_opt[0] * xx + b_opt) / w_opt[1]
plt.plot(xx, yy, 'g-', linewidth=3, label='Decision boundary')

# Margins
yy_margin_pos = -(w_opt[0] * xx + b_opt - 1) / w_opt[1]
yy_margin_neg = -(w_opt[0] * xx + b_opt + 1) / w_opt[1]
plt.plot(xx, yy_margin_pos, 'r--', linewidth=2, label='Margin (+1)')
plt.plot(xx, yy_margin_neg, 'b--', linewidth=2, label='Margin (-1)')

# Normal vector
origin = np.array([0, 0])
plt.arrow(origin[0], origin[1], w_opt[0], w_opt[1], 
         head_width=0.3, head_length=0.3, fc='green', ec='green', linewidth=2)
plt.text(w_opt[0] + 0.5, w_opt[1] + 0.5, 'w', fontsize=16, fontweight='bold')

plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('SVM Hard Margin: Support Vectors', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.axis('equal')
plt.xlim(-4, 4)
plt.ylim(-5, 5)
plt.show()

# KKT 조건 검증
print("\n=== KKT 조건 검증 ===")
print("\n1. Stationarity:")
print(f"   ∇_w L = {w_opt - np.sum(alpha_opt[:, np.newaxis] * y_train[:, np.newaxis] * X_train, axis=0)}")
print(f"   ∇_b L = {-np.sum(alpha_opt * y_train):.10f}")

print("\n2. Primal feasibility:")
for i in range(n_samples):
    margin = y_train[i] * (w_opt @ X_train[i] + b_opt)
    print(f"   y_{i}(w^T x_{i} + b) = {margin:.4f} >= 1: {margin >= 1 - 1e-6}")

print("\n3. Dual feasibility:")
for i in range(n_samples):
    print(f"   α_{i} = {alpha_opt[i]:.4f} >= 0: {alpha_opt[i] >= -1e-10}")

print("\n4. Complementary slackness:")
for i in range(n_samples):
    slack = y_train[i] * (w_opt @ X_train[i] + b_opt) - 1
    product = alpha_opt[i] * slack
    print(f"   α_{i} * [y_{i}(w^T x_{i} + b) - 1] = {product:.10f} ≈ 0: {abs(product) < 1e-6}")

## 7. 요약 및 핵심 공식

### 7.1 SVM Hard Margin 핵심 정리

| 항목 | 내용 |
|------|------|
| **목표** | Margin 최대화 |
| **Margin** | $\frac{1}{\\|w\\|}$ (한쪽), $\frac{2}{\\|w\\|}$ (전체) |
| **정규화** | $y_i(w^T x_i + b) \geq 1$ (Canonical form) |
| **Primal** | $\min \frac{1}{2}\\|w\\|^2$ s.t. $y_i(w^T x_i + b) \geq 1$ |
| **Dual** | $\max \sum_i \alpha_i - \frac{1}{2}\sum_{ij} \alpha_i \alpha_j y_i y_j x_i^T x_j$ |
| **Solution** | $w = \sum_i \alpha_i y_i x_i$ |
| **Support Vectors** | $\alpha_i > 0$ ⇔ $y_i(w^T x_i + b) = 1$ |

### 7.2 왜 Dual을 푸는가?

1. **Kernel trick**: Dual은 $x_i^T x_j$만 필요 → 커널로 대체 가능!
2. **Sparsity**: 대부분의 $\alpha_i = 0$ → 효율적
3. **Convexity**: QP solver로 글로벌 최적해 보장

### 7.3 한계

**Hard Margin의 문제점**:
- 선형 분리 가능해야만 작동
- Outlier에 매우 민감
- 실제 데이터는 대부분 **선형 분리 불가능**

→ 다음 노트북에서 **Soft Margin SVM**으로 해결!

## 8. 연습문제

### 문제 1: Margin 계산

다음 초평면과 점이 주어졌을 때:
- $w = [3, 4]^T$, $b = 0$
- $x_0 = [1, 2]^T$

1. 점 $x_0$에서 초평면까지의 거리를 계산하시오.
2. $w$를 정규화하여 margin이 1이 되도록 스케일하시오.

In [ ]:
# 문제 1 풀이

w = np.array([3, 4])
b = 0
x0 = np.array([1, 2])

# 1. 거리 계산
distance = np.abs(w @ x0 + b) / np.linalg.norm(w)
print("=== 문제 1 ===")
print(f"1. 거리 = |w^T x0 + b| / ||w|| = {distance:.4f}")

# 2. 정규화
# margin이 1이 되려면: |w^T x0 + b| = 1
# 현재: |w^T x0 + b| = {w @ x0 + b}
# 스케일: c = 1 / |w^T x0 + b|
scale = 1 / np.abs(w @ x0 + b)
w_normalized = scale * w
b_normalized = scale * b

print(f"\n2. 정규화:")
print(f"   원본: w = {w}, b = {b}")
print(f"   스케일 c = {scale:.4f}")
print(f"   정규화: w = {w_normalized}, b = {b_normalized}")
print(f"   검증: |w^T x0 + b| = {np.abs(w_normalized @ x0 + b_normalized):.4f}")
print(f"   Margin = 1 / ||w|| = {1 / np.linalg.norm(w_normalized):.4f}")

### 문제 2: Dual 유도 연습

Lagrangian에서 Dual을 유도하는 과정을 직접 계산하시오:

1. $\mathcal{L}(w, b, \alpha) = \frac{1}{2}||w||^2 - \sum_i \alpha_i [y_i(w^T x_i + b) - 1]$
2. $\frac{\partial \mathcal{L}}{\partial w} = 0$에서 $w$를 구하시오.
3. $\frac{\partial \mathcal{L}}{\partial b} = 0$에서 제약 조건을 구하시오.
4. 위의 결과를 $\mathcal{L}$에 대입하여 Dual objective를 유도하시오.

In [ ]:
# 문제 2 풀이 (수식 유도는 위 마크다운 참조)

print("=== 문제 2: Dual 유도 ===")
print("\n1. Lagrangian:")
print("   L(w, b, α) = (1/2)||w||² - Σ αᵢ[yᵢ(w^T xᵢ + b) - 1]")

print("\n2. ∂L/∂w = 0:")
print("   w - Σ αᵢ yᵢ xᵢ = 0")
print("   → w = Σ αᵢ yᵢ xᵢ")

print("\n3. ∂L/∂b = 0:")
print("   -Σ αᵢ yᵢ = 0")
print("   → Σ αᵢ yᵢ = 0")

print("\n4. Dual objective (대입 후):")
print("   L(α) = Σ αᵢ - (1/2)Σᵢⱼ αᵢ αⱼ yᵢ yⱼ xᵢ^T xⱼ")
print("   subject to: αᵢ >= 0, Σ αᵢ yᵢ = 0")

# 수치 예제로 검증
print("\n=== 수치 검증 ===")
# 앞서 구한 최적해 사용
print(f"최적 α: {alpha_opt}")
print(f"w = Σ αᵢ yᵢ xᵢ = {w_opt}")
print(f"Σ αᵢ yᵢ = {np.sum(alpha_opt * y_train):.10f} (should be 0)")

# Dual objective value
dual_val = np.sum(alpha_opt) - 0.5 * alpha_opt @ Q @ alpha_opt
primal_val = 0.5 * np.linalg.norm(w_opt)**2
print(f"\nDual objective = {-dual_val:.4f}")
print(f"Primal objective = {primal_val:.4f}")
print(f"Duality gap = {abs(primal_val - (-dual_val)):.10f} (should be 0)")

## 9. 추가 자료

### 참고 논문 및 교재

1. **강의 자료**:
   - ML_L14a_Constrained.Optimization_part1.pdf
   - ML_L14b_Constrained.Optimization_part2.pdf
   - ML_L15a_SVM_(linear-hard).pdf

2. **Online Resources**:
   - [MIT OpenCourseWare: Optimization](https://ocw.mit.edu/)
   - [Stanford CS229: SVM Notes](http://cs229.stanford.edu/notes/)

3. **YouTube**:
   - StatQuest: Support Vector Machines
   - 3Blue1Brown: Lagrange Multipliers

### 다음 단계

- 다음 노트북: **SVM Soft Margin** (Slack variables, C parameter)
- 추가 학습: Kernel SVM (ML_L15c_SVM_(kernel).pdf)